In [18]:
import os
import pandas as pd

import re
import time
import random
import string
from typing import Any

import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder


import matplotlib.pyplot as plt
%matplotlib inline

In [19]:
RANDOM_SEED = 42
ROOT_DIR = '../predictions/'

In [20]:
def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
seed_all(RANDOM_SEED)

In [21]:
train_embeddings = pd.read_csv(os.path.join(ROOT_DIR, 'train_embeddings_submission.csv'))
train_naive_bayes = pd.read_csv(os.path.join(ROOT_DIR, 'naive_bayes_laplace_normalised_train.csv'))
train_linear_regression = pd.read_csv(os.path.join(ROOT_DIR, 'train_linear_regression.csv'))

In [24]:
train_embeddings['Embeddings_Suspicious_Level'] = train_embeddings['Suspicious_Level']
train_embeddings = train_embeddings.drop(['Suspicious_Level'], axis=1)

In [28]:
train_naive_bayes = train_naive_bayes.drop(['Suspicious_Level'], axis=1)

In [31]:
train_linear_regression = train_linear_regression.drop(['Suspicious_Level'], axis=1)
train_linear_regression['Linear_Level1']=train_linear_regression['0']
train_linear_regression['Linear_Level2']=train_linear_regression['1']
train_linear_regression['Linear_Level3']=train_linear_regression['2']
train_linear_regression = train_linear_regression.drop(['0', '1', '2'], axis=1)

In [32]:
train_linear_regression.head()

,ChannelName,ChannelId,MessageId,Date,EditDate,Content,Sensitive Topic,num_sensitive,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3
0,boris_rozhin,1101806611,91626,2023-07-08 16:11:34,2023-07-08 16:11:47,Работа наших бойцов к югу от Артемовска. Работ...,"weapons,politics",2,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601
1,sashakots,1109403194,40853,2023-07-08 16:44:44,2023-07-08 16:44:58,"Анкара нарушила договорённости, отпустив глава...","politics,racism",2,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382
2,swodki,1144180066,280668,2023-07-09 02:00:23,2023-07-09 02:05:53,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,none,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347
3,boris_rozhin,1101806611,91573,2023-07-08 02:07:05,2023-07-08 02:07:19,МТ-ЛБ с 32-зарядной авиационной пусковой устан...,weapons,1,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918
4,swodki,1144180066,280695,2023-07-09 07:01:49,2023-07-09 07:05:08,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,none,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347


In [34]:
train_original = pd.read_csv('../datasets/train.csv')


In [37]:
total_train = pd.concat([train_embeddings, train_naive_bayes, train_linear_regression], axis=1)
total_train = total_train.drop_duplicates(subset='MessageId')

In [41]:
total_train.head()

,MessageId,Embeddings_Suspicious_Level,MessageId,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,ChannelName,ChannelId,MessageId,Date,...,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3,Suspicious_Level
0,91626,1,91626,0.000277,0.436488,0.000084,boris_rozhin,1101806611,91626,2023-07-08 16:11:34,...,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601,2
1,40853,3,40853,0.921367,0.000017,0.000014,sashakots,1109403194,40853,2023-07-08 16:44:44,...,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382,1
2,280668,2,280668,0.902208,0.000016,0.000016,swodki,1144180066,280668,2023-07-09 02:00:23,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1
3,91573,1,91573,0.412115,0.000089,0.000092,boris_rozhin,1101806611,91573,2023-07-08 02:07:05,...,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918,1
4,280695,2,280695,0.902208,0.000016,0.000016,swodki,1144180066,280695,2023-07-09 07:01:49,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1


In [40]:
total_train['Suspicious_Level'] = train_original['Suspicious_Level']

In [45]:
# Deduplicate based on 'MessageId'
total_train = total_train.drop_duplicates(subset='MessageId', keep='first')

In [46]:
total_train.head()

,MessageId,Embeddings_Suspicious_Level,MessageId,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,ChannelName,ChannelId,MessageId,Date,...,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3,Suspicious_Level
0,91626,1,91626,0.000277,0.436488,0.000084,boris_rozhin,1101806611,91626,2023-07-08 16:11:34,...,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601,2
1,40853,3,40853,0.921367,0.000017,0.000014,sashakots,1109403194,40853,2023-07-08 16:44:44,...,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382,1
2,280668,2,280668,0.902208,0.000016,0.000016,swodki,1144180066,280668,2023-07-09 02:00:23,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1
3,91573,1,91573,0.412115,0.000089,0.000092,boris_rozhin,1101806611,91573,2023-07-08 02:07:05,...,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918,1
4,280695,2,280695,0.902208,0.000016,0.000016,swodki,1144180066,280695,2023-07-09 07:01:49,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1


In [47]:
total_train.to_csv(os.path.join(ROOT_DIR, 'total_train.csv'), index=False)

In [48]:
total_train = total_train.drop(['MessageId'], axis=1)

In [49]:
total_train.head()

,Embeddings_Suspicious_Level,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,ChannelName,ChannelId,Date,EditDate,Content,Sensitive Topic,...,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3,Suspicious_Level
0,1,0.000277,0.436488,0.000084,boris_rozhin,1101806611,2023-07-08 16:11:34,2023-07-08 16:11:47,Работа наших бойцов к югу от Артемовска. Работ...,"weapons,politics",...,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601,2
1,3,0.921367,0.000017,0.000014,sashakots,1109403194,2023-07-08 16:44:44,2023-07-08 16:44:58,"Анкара нарушила договорённости, отпустив глава...","politics,racism",...,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382,1
2,2,0.902208,0.000016,0.000016,swodki,1144180066,2023-07-09 02:00:23,2023-07-09 02:05:53,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,none,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1
3,1,0.412115,0.000089,0.000092,boris_rozhin,1101806611,2023-07-08 02:07:05,2023-07-08 02:07:19,МТ-ЛБ с 32-зарядной авиационной пусковой устан...,weapons,...,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918,1
4,2,0.902208,0.000016,0.000016,swodki,1144180066,2023-07-09 07:01:49,2023-07-09 07:05:08,ЭТО ЕДИНСТВЕННЫЙ СПОСОБ ПОМОЧЬ НАМ! \n\nПополн...,none,...,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1


In [50]:
total_train = total_train.drop(['Content'], axis=1)

In [51]:
total_train.head()

,Embeddings_Suspicious_Level,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,ChannelName,ChannelId,Date,EditDate,Sensitive Topic,num_sensitive,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3,Suspicious_Level
0,1,0.000277,0.436488,0.000084,boris_rozhin,1101806611,2023-07-08 16:11:34,2023-07-08 16:11:47,"weapons,politics",2,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601,2
1,3,0.921367,0.000017,0.000014,sashakots,1109403194,2023-07-08 16:44:44,2023-07-08 16:44:58,"politics,racism",2,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382,1
2,2,0.902208,0.000016,0.000016,swodki,1144180066,2023-07-09 02:00:23,2023-07-09 02:05:53,none,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1
3,1,0.412115,0.000089,0.000092,boris_rozhin,1101806611,2023-07-08 02:07:05,2023-07-08 02:07:19,weapons,1,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918,1
4,2,0.902208,0.000016,0.000016,swodki,1144180066,2023-07-09 07:01:49,2023-07-09 07:05:08,none,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1


In [53]:
total_train = total_train.drop(['ChannelName', 'ChannelId', 'Date', 'EditDate', 'Sensitive Topic'], axis=1)

In [57]:
total_train.head()

,Embeddings_Suspicious_Level,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,num_sensitive,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,Linear_Level1,Linear_Level2,Linear_Level3,Suspicious_Level
0,1,0.000277,0.436488,0.000084,2,neutral,0.948454,neutral,0.745498,False,0.913780,0.059677,0.925722,0.014601,2
1,3,0.921367,0.000017,0.000014,2,neutral,0.498936,gratitude,0.868692,False,0.948626,0.907052,0.067565,0.025382,1
2,2,0.902208,0.000016,0.000016,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1
3,1,0.412115,0.000089,0.000092,1,neutral,0.973560,neutral,0.906539,False,0.940683,0.822248,0.121834,0.055918,1
4,2,0.902208,0.000016,0.000016,0,neutral,0.788881,gratitude,0.769272,False,0.949768,0.979169,0.011484,0.009347,1


In [55]:
total_train.to_csv(os.path.join(ROOT_DIR, 'filtered_total_train.csv'), index=False)

In [102]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming you have a DataFrame named 'df' with 15 columns
# The first 14 columns are your features, and the last column ('Suspicious_Level') is your target

# Separate features and target variable based on column names
target_column = 'Suspicious_Level'

X = total_train.drop(['Suspicious_Level'], axis=1)  # Features
X = X.drop(['Emotion'], axis=1)
X = X.drop(['num_sensitive'], axis=1)
# Sort columns alphabetically
X = X.reindex(sorted(X.columns), axis=1)
y = total_train['Suspicious_Level']  # Target variable

# Convert categorical columns (like 'Sentiment') to numerical using one-hot encoding
X = pd.get_dummies(X, columns=['Sentiment'], drop_first=True)
# X = pd.get_dummies(X, columns=['Emotion'], drop_first=True)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train)

# Create and train the RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

f1_ensemble_score = f1_score(y_test, y_pred, average='macro')
print(f'F1 Score: {f1_ensemble_score * 100:.2f}%')

# Display classification report
print('Classification Report:\n', classification_report(y_test, y_pred))


     Bayes_Suspicious_Level1  Bayes_Suspicious_Level2   
131                 0.999997                 0.000002  \
153                 0.308114                 0.000132   
77                  0.000137                 0.763460   
437                 0.000062                 0.931017   
255                 0.891699                 0.000030   
..                       ...                      ...   
71                  0.701555                 0.000059   
106                 0.911197                 0.003032   
270                 0.980798                 0.000004   
435                 0.735804                 0.000049   
102                 0.000037                 0.999963   

     Bayes_Suspicious_Level3  Dangerous  Dangerous_Score   
131             1.153699e-06      False         0.941817  \
153             1.162452e-04      False         0.935295   
77              1.537169e-04      False         0.937550   
437             2.305625e-05      False         0.952586   
255            

In [103]:
X.head()

,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,Dangerous,Dangerous_Score,Embeddings_Suspicious_Level,Emotion_Score,Linear_Level1,Linear_Level2,Linear_Level3,Sentiment_Score,num_sensitive,Sentiment_neutral,Sentiment_positive
0,0.000277,0.436488,0.000084,False,0.913780,1,0.745498,0.059677,0.925722,0.014601,0.948454,2,True,False
1,0.921367,0.000017,0.000014,False,0.948626,3,0.868692,0.907052,0.067565,0.025382,0.498936,2,True,False
2,0.902208,0.000016,0.000016,False,0.949768,2,0.769272,0.979169,0.011484,0.009347,0.788881,0,True,False
3,0.412115,0.000089,0.000092,False,0.940683,1,0.906539,0.822248,0.121834,0.055918,0.973560,1,True,False
4,0.902208,0.000016,0.000016,False,0.949768,2,0.769272,0.979169,0.011484,0.009347,0.788881,0,True,False


In [66]:
test_embeddings = pd.read_csv(os.path.join(ROOT_DIR, 'test_embeddings_submission.csv'))
test_naive_bayes = pd.read_csv(os.path.join(ROOT_DIR, 'naive_bayes_laplace_normalised_test.csv'))
test_linear_regression = pd.read_csv(os.path.join(ROOT_DIR, 'test_linear_regression.csv'))

In [70]:
test_embeddings.head()

,MessageId,Embeddings_Suspicious_Level
0,168641,3
1,36569,1
2,19697,1
3,49296,3
4,11784,1


In [68]:
test_embeddings['Embeddings_Suspicious_Level'] = test_embeddings['Suspicious_Level']
test_embeddings = test_embeddings.drop(['Suspicious_Level'], axis=1)

In [71]:
test_naive_bayes.head()

,MessageId,Suspicious_Level,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3
0,168641,1,0.003797,0.002835,0.000208
1,36569,1,0.010007,0.000267,0.000529
2,19697,1,0.998182,0.000510,0.000558
3,49296,1,0.089458,0.007711,0.010189
4,11784,1,0.372208,0.000410,0.000372


In [72]:
test_naive_bayes = test_naive_bayes.drop(['Suspicious_Level'], axis=1)

In [73]:
test_linear_regression.head()

,ChannelName,ChannelId,MessageId,Date,EditDate,Content,Sensitive Topic,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,num_sensitive,Suspicious_Level,0,1,2
0,ukraina_ru,1175084215,168641,2023-09-11 16:57:36,2023-09-11 16:57:50,"⚡️Администрация Байдена близка к тому, чтобы о...","weapons,politics",negative,0.608944,neutral,0.513087,False,0.954423,2,1,0.759211,0.146267,0.094522
1,bear007,1505866568,36569,2023-09-11 14:52:01,2023-09-11 14:52:12,РИА Новости публикует первые кадры с места гиб...,"offline_crime,politics",neutral,0.955055,neutral,0.714759,False,0.948009,2,1,0.343281,0.323890,0.332828
2,zakharprilepin,1217080686,19697,2023-09-11 08:26:09,NaN,Вице-спикер Госдумы Петр Толстой - о сроках ок...,"terrorism,politics",negative,0.850013,admiration,0.257196,False,0.954249,2,1,0.657900,0.012299,0.329801
3,denazi_ua,1783083983,49296,2023-09-11 12:30:37,2023-09-11 12:32:55,В Германии не осталось действующих специалисто...,weapons,neutral,0.769793,neutral,0.287832,False,0.955604,1,1,0.326257,0.457862,0.215881
4,spletnicca,1287596810,11784,2023-09-11 08:47:19,NaN,Военные медики рассказали правду о войне и её ...,politics,neutral,0.858099,curiosity,0.818388,False,0.958531,1,1,0.733633,0.208342,0.058025


In [74]:
test_linear_regression = test_linear_regression.drop(['Suspicious_Level'], axis=1)

In [75]:
test_linear_regression['Linear_Level1']=test_linear_regression['0']
test_linear_regression['Linear_Level2']=test_linear_regression['1']
test_linear_regression['Linear_Level3']=test_linear_regression['2']
test_linear_regression = test_linear_regression.drop(['0', '1', '2'], axis=1)

In [76]:
test_linear_regression.head()

,ChannelName,ChannelId,MessageId,Date,EditDate,Content,Sensitive Topic,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,num_sensitive,Linear_Level1,Linear_Level2,Linear_Level3
0,ukraina_ru,1175084215,168641,2023-09-11 16:57:36,2023-09-11 16:57:50,"⚡️Администрация Байдена близка к тому, чтобы о...","weapons,politics",negative,0.608944,neutral,0.513087,False,0.954423,2,0.759211,0.146267,0.094522
1,bear007,1505866568,36569,2023-09-11 14:52:01,2023-09-11 14:52:12,РИА Новости публикует первые кадры с места гиб...,"offline_crime,politics",neutral,0.955055,neutral,0.714759,False,0.948009,2,0.343281,0.323890,0.332828
2,zakharprilepin,1217080686,19697,2023-09-11 08:26:09,NaN,Вице-спикер Госдумы Петр Толстой - о сроках ок...,"terrorism,politics",negative,0.850013,admiration,0.257196,False,0.954249,2,0.657900,0.012299,0.329801
3,denazi_ua,1783083983,49296,2023-09-11 12:30:37,2023-09-11 12:32:55,В Германии не осталось действующих специалисто...,weapons,neutral,0.769793,neutral,0.287832,False,0.955604,1,0.326257,0.457862,0.215881
4,spletnicca,1287596810,11784,2023-09-11 08:47:19,NaN,Военные медики рассказали правду о войне и её ...,politics,neutral,0.858099,curiosity,0.818388,False,0.958531,1,0.733633,0.208342,0.058025


In [77]:
total_test = pd.concat([test_embeddings, test_naive_bayes, test_linear_regression], axis=1)
total_test = total_test.drop_duplicates(subset='MessageId')

In [78]:
total_test.head()

,MessageId,Embeddings_Suspicious_Level,MessageId,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,ChannelName,ChannelId,MessageId,Date,...,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,num_sensitive,Linear_Level1,Linear_Level2,Linear_Level3
0,168641,3,168641,0.003797,0.002835,0.000208,ukraina_ru,1175084215,168641,2023-09-11 16:57:36,...,negative,0.608944,neutral,0.513087,False,0.954423,2,0.759211,0.146267,0.094522
1,36569,1,36569,0.010007,0.000267,0.000529,bear007,1505866568,36569,2023-09-11 14:52:01,...,neutral,0.955055,neutral,0.714759,False,0.948009,2,0.343281,0.323890,0.332828
2,19697,1,19697,0.998182,0.000510,0.000558,zakharprilepin,1217080686,19697,2023-09-11 08:26:09,...,negative,0.850013,admiration,0.257196,False,0.954249,2,0.657900,0.012299,0.329801
3,49296,3,49296,0.089458,0.007711,0.010189,denazi_ua,1783083983,49296,2023-09-11 12:30:37,...,neutral,0.769793,neutral,0.287832,False,0.955604,1,0.326257,0.457862,0.215881
4,11784,1,11784,0.372208,0.000410,0.000372,spletnicca,1287596810,11784,2023-09-11 08:47:19,...,neutral,0.858099,curiosity,0.818388,False,0.958531,1,0.733633,0.208342,0.058025


In [79]:
total_test.to_csv(os.path.join(ROOT_DIR, 'total_test.csv'), index=False)

In [80]:
total_test = total_test.drop(['MessageId'], axis=1)

In [81]:
total_test = total_test.drop(['Content'], axis=1)

In [82]:
total_test = total_test.drop(['ChannelName', 'ChannelId', 'Date', 'EditDate', 'Sensitive Topic'], axis=1)

In [83]:
total_test.head()

,Embeddings_Suspicious_Level,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,Sentiment,Sentiment_Score,Emotion,Emotion_Score,Dangerous,Dangerous_Score,num_sensitive,Linear_Level1,Linear_Level2,Linear_Level3
0,3,0.003797,0.002835,0.000208,negative,0.608944,neutral,0.513087,False,0.954423,2,0.759211,0.146267,0.094522
1,1,0.010007,0.000267,0.000529,neutral,0.955055,neutral,0.714759,False,0.948009,2,0.343281,0.323890,0.332828
2,1,0.998182,0.000510,0.000558,negative,0.850013,admiration,0.257196,False,0.954249,2,0.657900,0.012299,0.329801
3,3,0.089458,0.007711,0.010189,neutral,0.769793,neutral,0.287832,False,0.955604,1,0.326257,0.457862,0.215881
4,1,0.372208,0.000410,0.000372,neutral,0.858099,curiosity,0.818388,False,0.958531,1,0.733633,0.208342,0.058025


In [109]:
# Convert categorical columns (like 'Sentiment') to numerical using one-hot encoding
X_test = pd.get_dummies(total_test, columns=['Sentiment'], drop_first=True)
X_test = X_test.drop(['Emotion'], axis=1)

X_test = X_test.reindex(sorted(X.columns), axis=1)
# X_test = pd.get_dummies(X_test, columns=['Emotion'], drop_first=True)

In [110]:
# Move 'num_sensitive' column from last to third-to-last position
column_order = list(X_test.columns)
column_order.insert(-2, column_order.pop())

# Reorder the columns
X_test = X_test[column_order]


In [111]:
X_test.head()

,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,Dangerous,Dangerous_Score,Embeddings_Suspicious_Level,Emotion_Score,Linear_Level1,Linear_Level2,Linear_Level3,Sentiment_Score,num_sensitive,Sentiment_neutral,Sentiment_positive
0,0.003797,0.002835,0.000208,False,0.954423,3,0.513087,0.759211,0.146267,0.094522,0.608944,2,False,False
1,0.010007,0.000267,0.000529,False,0.948009,1,0.714759,0.343281,0.323890,0.332828,0.955055,2,True,False
2,0.998182,0.000510,0.000558,False,0.954249,1,0.257196,0.657900,0.012299,0.329801,0.850013,2,False,False
3,0.089458,0.007711,0.010189,False,0.955604,3,0.287832,0.326257,0.457862,0.215881,0.769793,1,True,False
4,0.372208,0.000410,0.000372,False,0.958531,1,0.818388,0.733633,0.208342,0.058025,0.858099,1,True,False


In [112]:
X.head()

,Bayes_Suspicious_Level1,Bayes_Suspicious_Level2,Bayes_Suspicious_Level3,Dangerous,Dangerous_Score,Embeddings_Suspicious_Level,Emotion_Score,Linear_Level1,Linear_Level2,Linear_Level3,Sentiment_Score,num_sensitive,Sentiment_neutral,Sentiment_positive
0,0.000277,0.436488,0.000084,False,0.913780,1,0.745498,0.059677,0.925722,0.014601,0.948454,2,True,False
1,0.921367,0.000017,0.000014,False,0.948626,3,0.868692,0.907052,0.067565,0.025382,0.498936,2,True,False
2,0.902208,0.000016,0.000016,False,0.949768,2,0.769272,0.979169,0.011484,0.009347,0.788881,0,True,False
3,0.412115,0.000089,0.000092,False,0.940683,1,0.906539,0.822248,0.121834,0.055918,0.973560,1,True,False
4,0.902208,0.000016,0.000016,False,0.949768,2,0.769272,0.979169,0.011484,0.009347,0.788881,0,True,False


In [113]:
# Make predictions on the test set
y_test_pred = classifier.predict(X_test)

In [114]:
df_result = pd.DataFrame(y_test_pred, columns=['Suspicious_Level'])

In [115]:
df_result['MessageId'] = test_embeddings['MessageId']

In [116]:
df_result[['MessageId', 'Suspicious_Level']].to_csv('model_ensemble_test.csv', index=False)